## Downloading Pre-Trained Model

In [404]:
!wget https://zenodo.org/record/3987831/files/ResNet22_mAP%3D0.430.pth

--2022-03-29 01:28:02--  https://zenodo.org/record/3987831/files/ResNet22_mAP%3D0.430.pth
SSL_INIT
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 259135254 (247M) [application/octet-stream]
Saving to: ‘ResNet22_mAP=0.430.pth’

ResNet22_mAP=0.430. 100%[===================>] 247.13M  2.93MB/s    in 2m 3s   

2022-03-29 01:30:07 (2.02 MB/s) - ‘ResNet22_mAP=0.430.pth’ saved [259135254/259135254]



## Modify JSON

In [1]:
import json
import os

In [27]:
folder = 'result/threshold_detection/90_persen'

if os.path.exists(folder)==False:
    os.mkdir(folder)

In [28]:
folder = 'csv_files/threshold_detection/90_persen/data_01'
csv_train = os.path.join(folder,'train.csv')
csv_devel = os.path.join(folder,'devel.csv')
csv_test = os.path.join(folder,'test.csv')

dir_input = 'dataset/threshold_detection/90_persen'
dir_output = 'result/threshold_detection/90_persen'

if os.path.exists(dir_output)==False:
    os.mkdir(dir_output)

In [29]:
file_path = 'script/json/config.json'
with open(file_path, 'r+') as f:
    data = json.load(f)
    data['dataset']['train_csv'] = csv_train
    data['dataset']['eval_csv'] = csv_devel
    data['dataset']['test_csv'] = csv_test
    data['dataset']['train_data_root_path'] = dir_input
    data['dataset']['eval_data_root_path'] = dir_input
    data['dataset']['test_data_root_path'] = dir_input
    data['dataset']['class_balancer_batch'] = True
    data['train_config']['logs_path'] = dir_output
    
    data['model']['spec_aug'] = False
    data['data_aumentation']['insert_noise'] = False
    data['model']['mixup'] = True
    data['model']['mixup_alpha'] = 0.7
    
    data['model_name'] = 'panns'
    data['model']['pretreined_checkpoint'] = 'Cnn14_16k_mAP=0.438.pth'
    
    data['audio']['feature'] = 'melspectrogram'
    data['audio']['num_mels'] = 64
    data['audio']['hop_length'] = 320
    
    data['train_config']['optimizer'] = 'adamw'
    data['train_config']['learning_rate'] = 0.001
    data['train_config']['weight_decay'] = 0.001
    data['train_config']['batch_size'] = 16
    
    
    f.seek(0)        # <--- should reset file position to the beginning.
    json.dump(data,f,indent=4)
    f.truncate()     # remove remaining part

In [30]:
data

{'model_name': 'panns',
 'dataset': {'class_balancer_batch': True,
  'window_len': 3,
  'step': 1,
  'padding_with_max_lenght': True,
  'temporal_control': 'padding',
  'max_seq_len': None,
  'train_csv': 'csv_files/threshold_detection/90_persen/data_01/train.csv',
  'train_data_root_path': 'dataset/threshold_detection/90_persen',
  'eval_csv': 'csv_files/threshold_detection/90_persen/data_01/devel.csv',
  'eval_data_root_path': 'dataset/threshold_detection/90_persen',
  'test_csv': 'csv_files/threshold_detection/90_persen/data_01/test.csv',
  'test_data_root_path': 'dataset/threshold_detection/90_persen',
  'control_class': 0,
  'patient_class': 1},
 'model': {'pretreined_checkpoint': 'Cnn14_16k_mAP=0.438.pth',
  'freeze_base': False,
  'mixup': True,
  'mixup_alpha': 0.7,
  'spec_aug': False,
  'num_class': 1},
 'train_config': {'early_stop_epochs': 0,
  'lr_decay': True,
  'warmup_steps': 500,
  'epochs': 100,
  'learning_rate': 0.001,
  'weight_decay': 0.001,
  'optimizer': 'adamw'

In [31]:
with open(os.path.join(folder,'readme.txt')) as f:
    data = f.read()

print(data)

This File is CSV for Train, Devel, and Test
Train Data: 
--Compare Train
--Coughvid Positive Train
--Coswara Positive Train

Devel Data: 
--Compare Devel
--Coughvid Positive Devel
--Coswara Positive Devel

Devel Partision: 20.0%


## Training 5 Seeds

In [32]:
import os

config = file_path
seeds = [42]
for seed in seeds:
    command = f"python script/train.py -c {config} -s {seed}"
    os.system(command)

The Max Time dim Lenght is: 1502 (+- 30.04 seconds)
The Min Time dim Lenght is: 108 (+- 2.16 seconds)
Using Class Batch Balancer
Enable Mixup with alpha: 0.7
 > Partial model initialization.
 | > Layer missing in the model definition: spectrogram_extractor.stft.conv_real.weight
 | > Layer missing in the model definition: spectrogram_extractor.stft.conv_imag.weight
 | > Layer missing in the model definition: logmel_extractor.melW
 | > 81 / 81 layers are restored.
Partial Pretrained checkpoint loaded:  Cnn14_16k_mAP=0.438.pth
Starting new training run
Write summary at step 10  Loss:  0.988673210144043
Write summary at step 20  Loss:  0.7619819045066833
Write summary at step 30  Loss:  0.7722558975219727
Epoch 0 End !
Validation:
Acurracy:  0.2722222222222222 Acurracy Control:  0.967741935483871 Acurracy Patient:  0.12751677852348994 Acurracy Balanced 0.5476293570036804
Loss normal: 0.741675697432624 Loss Control: 0.6177162085810015 Loss Patient: 0.7674659278568805 Loss balanced:  0.69259

Validation:
Acurracy:  0.9166666666666666 Acurracy Control:  0.9032258064516129 Acurracy Patient:  0.9194630872483222 Acurracy Balanced 0.9113444468499675
Loss normal: 0.3160061127506196 Loss Control: 0.2643433514374098 Loss Patient: 0.3267547436083523 Loss balanced:  0.295549047522881 Loss1+loss2: 0.295549047522881
Write summary at step 460  Loss:  0.3067321181297302
Write summary at step 470  Loss:  0.2993176579475403
Write summary at step 480  Loss:  0.35116684436798096
Write summary at step 490  Loss:  0.5359079837799072
Epoch 12 End !
Validation:
Acurracy:  0.9 Acurracy Control:  0.7096774193548387 Acurracy Patient:  0.9395973154362416 Acurracy Balanced 0.8246373673955402
Loss normal: 0.3124019059869978 Loss Control: 0.4576820500435368 Loss Patient: 0.2821758361470779 Loss balanced:  0.3699289430953073 Loss1+loss2: 0.3699289430953073
Write summary at step 500  Loss:  0.6723043322563171
Saved checkpoint to: result/threshold_detection/90_persen/42/panns/checkpoint_500.pt
Validation:

Write summary at step 920  Loss:  0.19212502241134644
Write summary at step 930  Loss:  0.251920223236084
Write summary at step 940  Loss:  0.21968132257461548
Write summary at step 950  Loss:  0.30332106351852417
Epoch 24 End !
Validation:
Acurracy:  0.9166666666666666 Acurracy Control:  0.8387096774193549 Acurracy Patient:  0.9328859060402684 Acurracy Balanced 0.8857977917298117
Loss normal: 0.3146513349480099 Loss Control: 0.3141085966940849 Loss Patient: 0.31476424604454295 Loss balanced:  0.31443642136931393 Loss1+loss2: 0.31443642136931393
Write summary at step 960  Loss:  0.22066906094551086
Write summary at step 970  Loss:  0.41841670870780945
Write summary at step 980  Loss:  0.40577688813209534
Epoch 25 End !
Validation:
Acurracy:  0.9166666666666666 Acurracy Control:  0.8064516129032258 Acurracy Patient:  0.9395973154362416 Acurracy Balanced 0.8730244641697337
Loss normal: 0.284209292795923 Loss Control: 0.374382080570344 Loss Patient: 0.2654485066465083 Loss balanced:  0.31

Validation:
Acurracy:  0.9111111111111111 Acurracy Control:  0.7419354838709677 Acurracy Patient:  0.9463087248322147 Acurracy Balanced 0.8441221043515912
Loss normal: 0.2740199064215024 Loss Control: 0.4861879271845664 Loss Patient: 0.2298775553303277 Loss balanced:  0.35803274125744705 Loss1+loss2: 0.35803274125744705
Write summary at step 1410  Loss:  0.3191447854042053
Write summary at step 1420  Loss:  0.07831175625324249
Write summary at step 1430  Loss:  0.44955974817276
Write summary at step 1440  Loss:  0.500402569770813
Epoch 37 End !
Validation:
Acurracy:  0.9166666666666666 Acurracy Control:  0.7741935483870968 Acurracy Patient:  0.9463087248322147 Acurracy Balanced 0.8602511366096557
Loss normal: 0.2338977629939715 Loss Control: 0.3394639261307255 Loss Patient: 0.21193434428048613 Loss balanced:  0.27569913520560585 Loss1+loss2: 0.27569913520560585
Write summary at step 1450  Loss:  0.29160943627357483
Write summary at step 1460  Loss:  0.31885671615600586
Write summary at

Validation:
Acurracy:  0.9277777777777778 Acurracy Control:  0.8064516129032258 Acurracy Patient:  0.9530201342281879 Acurracy Balanced 0.8797358735657068
Loss normal: 0.2343848421341843 Loss Control: 0.47998526980800016 Loss Patient: 0.18328676537779354 Loss balanced:  0.33163601759289685 Loss1+loss2: 0.33163601759289685
Write summary at step 1870  Loss:  0.28511154651641846
Write summary at step 1880  Loss:  0.3930319547653198
Write summary at step 1890  Loss:  0.34581029415130615
Write summary at step 1900  Loss:  0.21423976123332977
Epoch 49 End !
Validation:
Acurracy:  0.9222222222222223 Acurracy Control:  0.8387096774193549 Acurracy Patient:  0.9395973154362416 Acurracy Balanced 0.8891534964277983
Loss normal: 0.27943643058339757 Loss Control: 0.38246802264644253 Loss Patient: 0.25800033373720693 Loss balanced:  0.32023417819182476 Loss1+loss2: 0.32023417819182476
Write summary at step 1910  Loss:  0.14383360743522644
Write summary at step 1920  Loss:  0.31965166330337524
Write s

Write summary at step 2320  Loss:  0.343784362077713
Write summary at step 2330  Loss:  0.3437480926513672
Write summary at step 2340  Loss:  0.1864379644393921
Write summary at step 2350  Loss:  0.26289188861846924
Epoch 61 End !
Validation:
Acurracy:  0.9222222222222223 Acurracy Control:  0.7419354838709677 Acurracy Patient:  0.959731543624161 Acurracy Balanced 0.8508335137475644
Loss normal: 0.24578294240766102 Loss Control: 0.6421990240773847 Loss Patient: 0.163307116535686 Loss balanced:  0.40275307030653534 Loss1+loss2: 0.40275307030653534
Write summary at step 2360  Loss:  0.29943859577178955
Write summary at step 2370  Loss:  0.27692654728889465
Write summary at step 2380  Loss:  0.3230093717575073
Write summary at step 2390  Loss:  0.3934566080570221
Epoch 62 End !
Validation:
Acurracy:  0.9333333333333333 Acurracy Control:  0.8387096774193549 Acurracy Patient:  0.9530201342281879 Acurracy Balanced 0.8958649058237713
Loss normal: 0.27310999168290034 Loss Control: 0.47879308462

Validation:
Acurracy:  0.9111111111111111 Acurracy Control:  0.7741935483870968 Acurracy Patient:  0.9395973154362416 Acurracy Balanced 0.8568954319116692
Loss normal: 0.2318909347885185 Loss Control: 0.4410672514669357 Loss Patient: 0.18837101917538868 Loss balanced:  0.3147191353211622 Loss1+loss2: 0.3147191353211622
Write summary at step 2820  Loss:  0.43977999687194824
Write summary at step 2830  Loss:  0.1264352947473526
Write summary at step 2840  Loss:  0.42663097381591797
Write summary at step 2850  Loss:  0.39924806356430054
Epoch 74 End !
Validation:
Acurracy:  0.9 Acurracy Control:  0.5806451612903226 Acurracy Patient:  0.9664429530201343 Acurracy Balanced 0.7735440571552284
Loss normal: 0.23021544449859196 Loss Control: 0.7064169222308744 Loss Patient: 0.13113997446610623 Loss balanced:  0.4187784483484903 Loss1+loss2: 0.4187784483484903
Write summary at step 2860  Loss:  0.22178654372692108
Write summary at step 2870  Loss:  0.4520326852798462
Write summary at step 2880  L

Write summary at step 3270  Loss:  0.1910688281059265
Write summary at step 3280  Loss:  0.3020232021808624
Write summary at step 3290  Loss:  0.22406670451164246
Write summary at step 3300  Loss:  0.17382991313934326
Epoch 86 End !
Validation:
Acurracy:  0.9222222222222223 Acurracy Control:  0.8064516129032258 Acurracy Patient:  0.9463087248322147 Acurracy Balanced 0.8763801688677202
Loss normal: 0.22338392130202717 Loss Control: 0.364692009264423 Loss Patient: 0.19398426599550567 Loss balanced:  0.27933813762996434 Loss1+loss2: 0.27933813762996434
Write summary at step 3310  Loss:  0.36584579944610596
Write summary at step 3320  Loss:  0.3986508250236511
Write summary at step 3330  Loss:  0.11777383089065552
Write summary at step 3340  Loss:  0.2725604176521301
Epoch 87 End !
Validation:
Acurracy:  0.9166666666666666 Acurracy Control:  0.7096774193548387 Acurracy Patient:  0.959731543624161 Acurracy Balanced 0.8347044814894999
Loss normal: 0.22963433116674423 Loss Control: 0.61150730

Validation:
Acurracy:  0.9055555555555556 Acurracy Control:  0.7096774193548387 Acurracy Patient:  0.9463087248322147 Acurracy Balanced 0.8279930720935267
Loss normal: 0.22138202827837733 Loss Control: 0.5781025886535645 Loss Patient: 0.14716500133876032 Loss balanced:  0.3626337949961624 Loss1+loss2: 0.3626337949961624
Write summary at step 3770  Loss:  0.4885949194431305
Write summary at step 3780  Loss:  0.22532959282398224
Write summary at step 3790  Loss:  0.31883642077445984
Write summary at step 3800  Loss:  0.39314132928848267
Epoch 99 End !
Validation:
Acurracy:  0.9055555555555556 Acurracy Control:  0.6774193548387096 Acurracy Patient:  0.9530201342281879 Acurracy Balanced 0.8152197445334488
Loss normal: 0.24790112351377805 Loss Control: 0.644174583496586 Loss Patient: 0.16545497221034644 Loss balanced:  0.4048147778534662 Loss1+loss2: 0.4048147778534662
------------------------------
SEED: 42 Best Loss: 0.23102611226226685
______________________________


In [33]:
import os

data = csv_test
folder = dir_input
experiment_dir = dir_output
musan = 'dataset/Musan-Data'
command = f"python script/test_all_seeds.py -t {data} -r {folder} --experiment_dir {experiment_dir}"
os.system(command)

DEBUG: 1
Not K-Fold Experiment
Return Potential: False
Loading checkpoint: result/threshold_detection/90_persen/42/panns/best_checkpoint.pt
Model Sucessful Load !
++++++++++++++++++++++++++++++++++++++++
Control Files Classified incorrectly:
Num. Files: 1
['test_033.wav']
++++++++++++++++++++++++++++++++++++++++
----------------------------------------
Patient Files Classified incorrectly:
Num. Files: 3
['test_046.wav' 'test_043.wav' 'test_067.wav']
----------------------------------------
======== Confusion Matrix ==========
Predicted  0.0  1.0  All
Target                  
0           39    1   40
1            3    8   11
All         42    9   51
Test
  Acurracy:  0.9215686274509803 Acurracy Control:  0.975 Acurracy Patient:  0.7272727272727273
F1: 0.7999999999999999 UAR: 0.8511363636363636


0

In [34]:
import glob

files = glob.glob(os.path.join(dir_output,'*/*/*.pt'))

for file in files:
    filename = file.split('/')[-1]
    if filename.split('_')[0] == 'checkpoint':
        os.remove(file)